# Init OpenSky into Kafka

In [ ]:
import requests
from kafka import KafkaProducer
import json

correspond = ["icao24","callsign","origin_country","time_position",
              "last_contact","longitude","latitude","baro_altitude"
              ,"on_ground","velocity","true_track","vertical_rate",
              "sensors","geo_altitude","squawk","spi","position_source","category"]



def send_opensky_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from ThingSpeak
    
    response = requests.get(url)

    # Check if the request was successful

    if response.status_code == 200:
        data = response.json()
        if data and "states" in data:
            for state in data["states"]:
                flight_info = {key: state[i] for i, key in enumerate(correspond) if key}
                producer.send(topic, value=flight_info)
            producer.flush()
            print(f"Sent {len(data['states'])} records to Kafka.")
        else:
            print("No valid data to send.")

## Initiate beginning

In [ ]:
from datetime import datetime, timedelta

api_url = "https://opensky-network.org/api/states/all"

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)

start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

## Exemple with 10 calls.

In [ ]:
import time 
start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

print(f"{start} - {end}")
send_opensky_to_kafka("opensky", api_url, fields)

time.sleep(5)

# Update time variables
initial_date = end_date
end_date = initial_date + timedelta(minutes=step)

2025-01-25 16:00:00 - 2025-01-25 17:00:00
Sent 6715 records to Kafka.
['e49406', 'GLO1614 ', 'Brazil', 1742295233, 1742295236, -45.7226, -22.3238, 9448.8, False, 249.14, 35.76, 0.98, None, 10050.78, None, False, 0]
